In [34]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta


In [3]:

from config import ACCESS_KEY, SECRET_KEY, TOKEN

In [55]:
def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone)) - datetime.timedelta(days=1)
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)
    for_last_month = yesterday - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
            'last_month':for_last_month.strftime('%m')

            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']
last_month = get_now_datetime_str()['now_date']

limit = 100
offset = 1

In [5]:
now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']

In [6]:
token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3
BUCKET_NAME = 'dwh-asgard' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

In [8]:
def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'



def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [9]:
FOLDER = 'all_intercoms_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [15]:
FOLDER = 'revenue_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [16]:
FOLDER = 'number_of_services_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [47]:
FOLDER = 'all_flats_rep_partners'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [18]:
FOLDER = 'conected_intercoms_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [19]:
FOLDER = 'address_flats_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [20]:
FOLDER = 'count_of_online_intercoms_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [21]:
FOLDER = 'separate_cameras_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [23]:
FOLDER = 'count_of_instaletion_point_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [24]:
FOLDER = 'arhive_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [25]:
FOLDER = 'partners_count_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [26]:
FOLDER = 'all_partners_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [27]:
FOLDER = 'arhive_enterprise_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [28]:
FOLDER = 'enterprice_commerce_intercom_count_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [29]:
FOLDER = 'intercom_uuid_count_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [30]:
FOLDER = 'accruals_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [31]:
FOLDER = 'count_of_blocks_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [32]:
FOLDER = 'count_of_io_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [37]:
FOLDER =  'revenue_commercial_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False)
remove_temp_files()

In [39]:
FOLDER =  'number_of_services_commercial_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [40]:
FOLDER =  'accruals_commercial_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [41]:
FOLDER =  'partners_count_enterprice_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [42]:
FOLDER =  'revenue_enterpise_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [43]:
FOLDER =  'number_of_services_enterprise_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [44]:
FOLDER =  'accruals_enterprise_rep_pertner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

___

In [56]:
year_date = pd.DataFrame({
    'year':[f'{year}']
})

In [58]:
year_date

,year
0,2024


In [59]:
month_date = pd.DataFrame({
    'month':[f'{month}']
})

In [60]:
month_date

,month
0,04


In [63]:
ACCESS_KEY = ACCESS_KEY
SECRET_KEY = SECRET_KEY
BUCKET_NAME = "dwh-asgard"
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'C:/Users/Boris/Desktop/report_partner/ready_report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"partner_report/year={int(year)}/{int(month)}.csv"
    )

year_date = pd.DataFrame({
    'year':[f'{year}']
})

month_date = pd.DataFrame({
    'month':[f'{month}']
})

partners_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partners_count_rep_partner/{int(year)}/{int(month)}.csv')
address_flats_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/address_flats_rep_partner/{int(year)}/{int(month)}.csv')
all_partners_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_partners_rep_partner/{int(year)}/{int(month)}.csv')
arhive_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/arhive_rep_partner/{int(year)}/{int(month)}.csv')
conected_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/conected_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
count_of_instaletion_point_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_instaletion_point_rep_partner/{int(year)}/{int(month)}.csv')
count_of_online_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_online_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
separate_cameras_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/separate_cameras_rep_partner/{int(year)}/{int(month)}.csv')
all_flats_rep_partners = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_flats_rep_partners/{int(year)}/{int(month)}.csv')
revenue_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_rep_partner/{int(year)}/{int(month)}.csv')
all_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
arhive_enterprise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/arhive_enterprise_rep_partner/{int(year)}/{int(month)}.csv')
enterprice_commerce_intercom_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/enterprice_commerce_intercom_count_rep_partner/{int(year)}/{int(month)}.csv')
intercom_uuid_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/intercom_uuid_count_rep_partner/{int(year)}/{int(month)}.csv')
partners_count_enterprice_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partners_count_enterprice_rep_partner/{int(year)}/{int(month)}.csv')
revenue_enterpise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_enterpise_rep_partner/{int(year)}/{int(month)}.csv')
revenue_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_commercial_rep_partner/{int(year)}/{int(month)}.csv')
accruals_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/accruals_rep_partner/{int(year)}/{int(month)}.csv')
accruals_enterprise_rep_pertner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/accruals_enterprise_rep_pertner/{int(year)}/{int(month)}.csv')
accruals_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/accruals_commercial_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_commercial_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_enterprise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_enterprise_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_rep_partner/{int(year)}/{int(month)}.csv')
count_of_blocks_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_blocks_rep_partner/{int(year)}/{int(month)}.csv')
count_of_io_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_io_rep_partner/{int(year)}/{int(month)}.csv')
partner_programm_amount_rep_partner  = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partner_programm_amount_rep_partner/{int(year)}/{int(month)}.csv')
    
concated = pd.concat([
    year_date,
    month_date,
    partners_count_rep_partner,
    address_flats_rep_partner,
    all_partners_rep_partner,
    arhive_rep_partner,
    conected_intercoms_rep_partner,
    count_of_instaletion_point_rep_partner,
    count_of_online_intercoms_rep_partner,
    separate_cameras_rep_partner,
    all_flats_rep_partners,
    revenue_rep_partner,
    all_intercoms_rep_partner,
    arhive_enterprise_rep_partner,
    enterprice_commerce_intercom_count_rep_partner,
    intercom_uuid_count_rep_partner,
    partners_count_enterprice_rep_partner,
    revenue_enterpise_rep_partner,
    revenue_commercial_rep_partner,
    accruals_rep_partner,
    accruals_enterprise_rep_pertner,
    accruals_commercial_rep_partner,
    number_of_services_commercial_rep_partner,
    number_of_services_enterprise_rep_partner,
    number_of_services_rep_partner,
    count_of_blocks_rep_partner,
    count_of_io_rep_partner,
    partner_programm_amount_rep_partner
    ]
    , axis=1   
    )

concated.to_csv(f'C:/Users/Boris/Desktop/report_partner/ready_report/{int(year)}/{int(month)}.csv', sep=',', index=False)
    
get_s3_instance()
upload_dump_to_s3()
